In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import shutil
import itertools
import pathlib
from PIL import Image
#Data handling tools
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('whitegrid')
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , classification_report

#Deep learning libs
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D , MaxPooling2D , Flatten , Activation , Dense , Dropout , BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam , Adamax
from tensorflow.keras import regularizers

#Warningds
import warnings
warnings.filterwarnings('ignore')

**Reading of Training Dataset into a Dataframe**

In [ ]:
train_data_path = '/content/drive/MyDrive/vision project/dataset/train'

filepaths =[]
labels = []

folds = os.listdir(train_data_path)

for fold in folds:
    f_path = os.path.join(train_data_path , fold)
    filelists = os.listdir(f_path)

    for file in filelists:
        filepaths.append(os.path.join(f_path , file))
        labels.append(fold)

#Concat data paths with labels
Fseries = pd.Series(filepaths , name = 'filepaths')
Lseries = pd.Series(labels , name = 'label')
train_df = pd.concat([Fseries , Lseries] , axis = 1)

In [ ]:
train_df

**Reading of Testing data into a Dataframe**

In [ ]:
test_data_path = '/content/drive/MyDrive/vision project/dataset/test'
filepaths =[]
labels = []

folds = os.listdir(test_data_path)

for fold in folds:
    f_path = os.path.join(test_data_path , fold)
    filelists = os.listdir(f_path)

    for file in filelists:
        filepaths.append(os.path.join(f_path , file))
        labels.append(fold)

#Concat data paths with labels
Fseries = pd.Series(filepaths , name = 'filepaths')
Lseries = pd.Series(labels , name = 'label')
test_df = pd.concat([Fseries , Lseries] , axis = 1)

In [ ]:
test_df

**Splinting testing data to validation and test**

In [ ]:
valid,test = train_test_split(test_df,train_size=0.5,shuffle=True,random_state=42)

**Data Preprocessing**

in this part an image data generator will be applied to maximize the size of the dataset and set images size to (244,244)

this will be applied for train, test, and valisation data

In [ ]:
img_size = (180 ,180)
batch_size = 16

tr_gen = ImageDataGenerator (width_shift_range=0.3, height_shift_range=0.3)
ts_gen= ImageDataGenerator(width_shift_range=0.3, height_shift_range=0.3)

train_gen = tr_gen.flow_from_dataframe(train_df , x_col = 'filepaths' , y_col = 'label' , target_size = img_size ,
                                      class_mode = 'categorical' , color_mode = 'rgb' , shuffle = True , batch_size =batch_size)

valid_gen = ts_gen.flow_from_dataframe(valid , x_col = 'filepaths' , y_col = 'label' , target_size = img_size ,
                                       class_mode = 'categorical',color_mode = 'rgb' , shuffle= True, batch_size = batch_size)

test_gen = ts_gen.flow_from_dataframe(test , x_col= 'filepaths' , y_col = 'label' , target_size = img_size ,
                                      class_mode = 'categorical' , color_mode= 'rgb' , shuffle = False , batch_size = batch_size)

**Viewing a sample of the taring data**

In [ ]:
gen_dict = train_gen.class_indices
classes = list(gen_dict.keys())
images , labels = next(train_gen)

plt.figure(figsize= (20,20))

for i in range(16):
    plt.subplot(4,4,i+1)
    image = images[i] / 255
    plt.imshow(image)
    index = np.argmax(labels[i])
    class_name = classes[index]
    plt.title(class_name , color = 'blue' , fontsize= 12)
    plt.axis('off')
plt.show();

**Model building**

The model architecture have been developed as following:

First-> for the convolutional layers and pooling layers the EfficientNetB3 architecture is used

(feature extraction)

Second-> the data will be normalized for more efficient training of data and faster processing.

Third-> a fully connected layer (dens) will be applied with 256 neurons.

Forth-> a dropout function will be applied with a value of 0.4 to avoid overfitting

Finally-> the output layer (fully connected layer(dens)) will be appied with 5 neurons (numbeer of calsses).

In [ ]:
img_shape = (img_size[0],img_size[1],3)
num_class = len(classes)

base_model = tf.keras.applications.efficientnet.EfficientNetB3(include_top = False , weights = 'imagenet' ,
                                                               input_shape = img_shape, pooling= 'max')
model = Sequential([
    base_model,
    BatchNormalization(axis= -1 , momentum= 0.99 , epsilon= 0.001),
    Dense(256, activation = 'relu'),
    Dropout(0.4),
    Dense(num_class , activation = 'softmax')
])

model.compile(Adamax(learning_rate = 0.001) , loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

**Model Training**

In [ ]:
history = model.fit(x= train_gen , epochs = 11 , validation_data = valid_gen)

**Model Performance Visualization**

In [ ]:
# Assuming 'history' is the object returned by model.fit

# Plot training and validation accuracy over epochs
plt.figure(figsize=(12, 4))

# Plot training accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot training and validation loss over epochs
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

**Model Evaluation**

In [ ]:
train_score = model.evaluate(train_gen , steps =16 , verbose = 1)
valid_score = model.evaluate(valid_gen , steps = 16 , verbose = 1)
test_score = model.evaluate(test_gen , steps = 16 , verbose = 1)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 40)
print("Validation Loss: ", valid_score[0])
print("Validation Accuracy: ", valid_score[1])
print('-' * 40)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

**Measurment of the Model Accuracy and Viewing of Classification report**

In [ ]:
# Assuming model and test_images, test_labels are defined

# Make predictions on the test set
preds = model.predict_generator(test_gen)
y_pred = np.argmax(preds,axis=1)

# Display classification report
print("Classification Report:\n", classification_report(test_gen.classes, y_pred))

# Display confusion matrix
conf_matrix = confusion_matrix(test_gen.classes, y_pred)
print("Confusion Matrix:\n", conf_matrix)

#claucalting TP,TF,NP, and NF values
total_TP = 0
total_TN = 0
total_FP = 0
total_FN = 0

for i in range(conf_matrix.shape[0]):
    TP = conf_matrix[i, i]  # True Positives (diagonal element)

    # False Positives: sum of column i excluding TP
    FP = np.sum(conf_matrix[:, i]) - TP

    # False Negatives: sum of row i excluding TP
    FN = np.sum(conf_matrix[i, :]) - TP

    # True Negatives: Everything else outside the particular row and column
    TN = np.sum(conf_matrix) - TP - FP - FN

    total_TP += TP
    total_TN += TN
    total_FP += FP
    total_FN += FN

# Calculate and print accuracy, precision, recall, and F1-score
accuracy = (total_TP + total_TN)/(total_TP + total_TN + total_FN + total_FP)
precision = total_TP / (total_TP + total_FP)
recall = TP / (total_TP + total_FN)
f1_score = 2 * (precision * recall) / (precision + recall)


'''print("   T\t F\nT ",total_TP,"\t",total_TN)
print("\nF ",total_FN,"\t",total_FP)'''
print("Accuracy:", accuracy * 100)
print("Precision:", precision *100)
print("Recall:", recall *100)
print("F1-Score:", f1_score *100)


**Confusion Matrix**

In [ ]:
# Define class names
class_names = ['18-20','21-30','31-40','41-50','51-60']

# Plot the confusion matrix with numbers
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Oranges)
plt.title('Confusion Matrix')
plt.colorbar()

# Add class names to ticks
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names, rotation=45)
plt.yticks(tick_marks, class_names)

# Add numbers to cells
for i in range(len(class_names)):
    for j in range(len(class_names)):
        plt.text(j, i, str(conf_matrix[i, j]), ha='center', va='center', color='Blue', fontweight='bold')

plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

In [ ]:
from tensorflow.keras.models import load_model

model.save('/content/drive/MyDrive/vision project/AgeClassificationModel.hdf5')